Wolt Summer 2019 Internship Assignment


example of how to run program in the command line:

median_times('Helsinki',"13-01-19",19,20)

the output would then be found in a csv file named HelsinkiMedian_Pickup_Times.csv


Please make sure the locations.csv and pickup_times.csv files are located in the same folder as this notebook in order for the files to be properly imported and used by the program. 

In [1]:
import pandas as pd
import dateutil
import csv
from datetime import datetime


def median(lst):
    """Function to find the median value in a given list"""
    ### This function will be used to find the median pick up time of each location
    
    n = len(lst)
    if n < 1:
            return None
    if n % 2 == 1:
            return sorted(lst)[n//2]
    else:
            return sum(sorted(lst)[n//2-1:n//2+1])/2.0


def converter(s, format_list, format_output):
    """Function to convert from ISO format to datetime format"""
    for date_format in format_list:
        try:
            return datetime.strptime(s, date_format).strftime(format_output)
        except ValueError:
            continue      
    return 'incorrect date format'


def median_times(name_Of_City,pickup_date,startTime,endTime):
    """ Function calculates median pickup times for each location 
        name_Of_City=enter name of city in string format for example "Helsinki"
        pickup_date= date of pick ups in european format DD-MM-YY for example "17-02-2019"
        startTime= starting hour using a 24 hour clock example imput : 19
        endTime= ending hour using a 24 hour clock example imput : 20 """
    
    ### Variables,Lists,and dictionaries
    city_String=str(name_Of_City)+"Median_Pickup_Times.csv"
    datetime_List=[]
    median_Pickup_Times_List=[]
    median_Dict=dict()

    
    ### CSV imports
    pickup_Times_Df= pd.read_csv("pickup_times.csv")
    locations_Df= pd.read_csv("locations.csv")


    ### change iso format to datetime format using dateutil parser
    for i in range(len(pickup_Times_Df["iso_8601_timestamp"])):
        
        datetime_List.append(dateutil.parser.parse(pickup_Times_Df["iso_8601_timestamp"][i]))
    

    ### create a dictionary with the new datetime formatted time and convert it to a pandas DataFrame
    datetimeDict={'location_id' : pickup_Times_Df['location_id'], 'datetime_timestap' : datetime_List, 'pickup_time' : pickup_Times_Df['pickup_time']}
    datetime_Pickup_Times_Df=pd.DataFrame(datetimeDict)


    ### Set the index of the new Datetime pickup times DataFrame to be the datetime of each row.
    datetime_Pickup_Times_Df['datetime'] = pd.to_datetime(datetime_Pickup_Times_Df['datetime_timestap'])
    datetime_Pickup_Times_Df = datetime_Pickup_Times_Df.set_index('datetime')
    datetime_Pickup_Times_Df.drop(['datetime_timestap'], axis=1, inplace=True)
    
    
    ### Create a dictionary where the keys are the location ID's of each restaraunt location
    for i in range(len(locations_Df)):
        
        median_Dict[locations_Df['location_id'][i]]=[]
    
    
    ### convert the input date and time into desired format simple check incase a format like 17-02-19 is entered it will turn it into 17-02-2019
    if len(pickup_date)< 10:
        
        pickup_date=pickup_date[0:6]+"20"+pickup_date[6:len(pickup_date)]
    
    
    #Call the converter function to convert the user given input into the correct date format
    pickupStr = converter(str(pickup_date),format_list=['%d-%m-%Y', '%m-%d-%Y', '%Y-%d-%m',],format_output='%Y-%m-%d')
    
    
    #create the start and end date strings
    startDate= pickupStr +" " +str(startTime)+":00:00"
    endDate= pickupStr +" " +str(endTime)+":00:00"
    
    
    #Create a new Data Frame that contains only pick ups that occur within the inputted date and time
    date_Range_Df = datetime_Pickup_Times_Df[(datetime_Pickup_Times_Df.index >= startDate) & (datetime_Pickup_Times_Df.index <= endDate)]
    
    
    ###Create a dictionary containing the location IDs as the key with each key pointing to all pickup times for that location ID
    for i in range(len(date_Range_Df['location_id'])):
        
        median_Dict[pickup_Times_Df['location_id'][i]].append(date_Range_Df['pickup_time'][i])
    
    
    ### Create a list of lists containing the location ID and the median pick up time for that location in each list 
    for i in range(1,len(median_Dict.keys())):
        
        median_Pickup_Times_List.append([i,median(median_Dict[i])])
    
    
    
    
    ### write the median pickup times list to a new csv file named based on the city input given by the user 
    with open(city_String, 'w',newline='') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(["Location_ID", "Median Pickup Time"])
        writer.writerows(median_Pickup_Times_List)

    csvFile.close()